# PART 5: Regression Analysis

## Multiple Regression

### The Movie Dataset - Preparing the Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
movie = pd.read_csv("movies_metadata.csv", low_memory= False)

In [ ]:
movie.head()

In [ ]:
movie.info()

In [ ]:
movie = movie.set_index(pd.to_datetime(movie.release_date, errors = "coerce")).drop(columns = ["release_date"])

In [ ]:
movie.sort_index(inplace = True)
movie

In [ ]:
# Remove rows with NaT in the index
movie = movie[~movie.index.isna()]
movie

In [ ]:
df = movie.loc["2010":"2016", ["title", "budget", "revenue", "runtime", "vote_average", "popularity"]].copy()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.budget = pd.to_numeric(df.budget, errors = "coerce")

In [ ]:
df.popularity = pd.to_numeric(df.popularity, errors = "coerce")

In [ ]:
df = df[(df.revenue > 0) & (df.budget > 0)]

In [ ]:
df

In [ ]:
df.set_index("title", inplace = True)

In [ ]:
df.info()

In [ ]:
df.loc[:, ["budget", "revenue"]] = df.loc[:, ["budget", "revenue"]] / 1000000

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
sns.pairplot(df, kind = "reg")
plt.show()

### Multiple Regression Analysis with statsmodels

In [ ]:
df

In [ ]:
df.info()

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
model = ols("revenue ~ budget + runtime + vote_average + popularity", data = df)

In [ ]:
results = model.fit()

In [ ]:
results.params

In [ ]:
results.predict()

In [ ]:
df.insert(loc = 2, column = "predict", value = results.predict())

In [ ]:
df

In [ ]:
df.revenue - df.predict

In [ ]:
results.resid

In [ ]:
df = df.sort_values("revenue").reset_index()

In [ ]:
df

In [ ]:
df.loc[:, ["revenue", "predict"]].plot(figsize = (15, 10))
plt.legend(fontsize = 15)
plt.show()

In [ ]:
results.ssr

In [ ]:
results.mse_resid

In [ ]:
results.mse_resid**0.5

### Coefficient of Determination: the Adjusted R squared

In [ ]:
results.rsquared

In [ ]:
results.rsquared_adj

### Regression Coefficients, Hypothesis Testing & Model Specification

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
df

In [ ]:
results.params

In [ ]:
results.tvalues

In [ ]:
results.pvalues

In [ ]:
print(results.summary())

In [ ]:
sns.regplot(x = "runtime", y = "revenue", data = df)
plt.show()

In [ ]:
sns.regplot(x = "budget", y = "runtime", data = df)
plt.show()

In [ ]:
model = ols("revenue ~  budget + vote_average + popularity", data = df)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

### The F-Test

In [ ]:
results.fvalue

In [ ]:
results.f_pvalue

In [ ]:
print(results.summary())

### Creating and working with Dummy Variables (Part 1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
movie = pd.read_csv("movies_metadata.csv", low_memory= False)

In [ ]:
movie = movie.set_index(pd.to_datetime(movie.release_date, errors = "coerce")).drop(columns = ["release_date"])

In [ ]:
movie.sort_index(inplace = True)

In [ ]:
# Remove rows with NaT in the index
movie = movie[~movie.index.isna()]
movie

In [ ]:
movie

In [ ]:
df = movie.loc["2010":"2016", ["title", "budget", "revenue", "vote_average", "popularity",
                              "belongs_to_collection", "original_language"]].copy()

In [ ]:
df.budget = pd.to_numeric(df.budget, errors = "coerce")

In [ ]:
df.popularity = pd.to_numeric(df.popularity, errors = "coerce")

In [ ]:
df = df[(df.revenue > 0) & (df.budget > 0)]

In [ ]:
df.loc[:, ["budget", "revenue"]] = df.loc[:, ["budget", "revenue"]] / 1000000

In [ ]:
df.sort_values("revenue", ascending = False)

In [ ]:
df.belongs_to_collection.notnull()

In [ ]:
df.belongs_to_collection = df.belongs_to_collection.notnull()

In [ ]:
df.set_index("title", inplace = True)

In [ ]:
df

In [ ]:
plt.figure(figsize = (12,8))
sns.set(font_scale=1.5)
sns.regplot(x = "belongs_to_collection", y = "revenue", data = df)
plt.show()

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
model = ols("revenue ~ budget + popularity + vote_average + belongs_to_collection", data = df)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

### Creating and working with Dummy Variables (Part 2)

In [ ]:
df

In [ ]:
df.original_language.nunique()

In [ ]:
df.original_language.value_counts()

In [ ]:
df_dumm = pd.get_dummies(df, columns = ["original_language"])

In [ ]:
df_dumm

In [ ]:
df_dumm.columns.str.contains("language").sum()

In [ ]:
plt.figure(figsize = (12,8))
sns.set(font_scale=1.5)
sns.regplot(x = "original_language_en", y = "revenue", data = df_dumm)
plt.show()

In [ ]:
model = ols("revenue ~ original_language_en", data = df_dumm)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

In [ ]:
model = ols("revenue ~ budget + popularity + vote_average + belongs_to_collection", data = df_dumm)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())